In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tensorflow as tf

from skimage import color, exposure, filters, io, morphology, util
from math import sqrt
from numpy import loadtxt
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

### A. _Fully Connected Neural Networks_
1. Lakukan klasifikasi untuk dataset cifar menggunakan **_Fully Connected Neural Networks_** dengan arsitektur dasar sebagai berikut:
    - _Fully Connected Layer_ ($1000$ _hidden node_).
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Fully Connected Layer_ ($1000$ _hidden node_)
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Output Layer_ (10 kelas).
    - _Activation Layer_ ($\text{Softmax function}$).
    - _Classification Result_.

Fitur yang digunakan bebas (_handcrafted_/_non-convolution_). Hitung metrik ($\text{accuracy}$, $\text{sensitivity}$, $\text{specificity}$, dan $f_{1}\text{ score}$) untuk data train dan data test.

#### Mendefinisikan Fungsi Bantuan
Untuk memudahkan implementasi, berikut beberapa fungsi bantuan yang akan dipakai

##### Menghitung $\text{recall}$ atau $\text{sensitivity}$
$$\text{recall} = \frac{t_p}{t_p+t_n}$$

In [2]:
def recall(y_true, y_pred): 
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  return true_positives / (possible_positives + K.epsilon())

##### Menghitung $\text{precision}$
$$\text{precision} = \frac{t_p}{t_p + f_p}$$

In [3]:
def precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  return true_positives / (predicted_positives + K.epsilon())

##### Menghitung $\text{specificity}$
$$\text{specificity} = \frac{t_n}{t_n + f_p}$$

In [4]:
def specificity(y_true, y_pred):
  true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
  possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
  return true_negatives / (possible_negatives + K.epsilon())

##### Menghitung $f_{1}\text{ Score}$
Kita tau bahwa
$$f_{\beta} = (1 + \beta^{2}) \cdot \frac{\text{precition} \cdot \text{recall}}{(\beta^{2} \cdot \text{precision}) + \text{recall}}$$
Sehingga 
$$f_{1} = (1 + 1^{2}) \cdot \frac{\text{precition} \cdot \text{recall}}{(1^{2} \cdot \text{precision}) + \text{recall}}$$
$$f_{1} = (2) \cdot \frac{\text{precition} \cdot \text{recall}}{(\text{precision}) + \text{recall}}$$



In [5]:
def f1_score(y_true, y_pred):
  precision_val = precision(y_true, y_pred)
  recall_val = recall(y_true, y_pred)
  return 2*((precision_val * recall_val)/(precision_val + recall_val + K.epsilon()))

**n.b.: Semua fungsi pada bagian penyebut ditambah epsilon untuk menghindari dibagi $0$ dikarenakan memasukkan input sebanyak $0$**

In [25]:
def predict_and_print_metrics(model, x_test, y_test):
  _, accuracy, sensitivity, specificity, score_f1 = model.evaluate(x_test, y_test, verbose=0)
  print("Accuracy: %.9f" % (accuracy))
  print("Sensitivity: %.9f" % (sensitivity))
  print("Specificity: %.9f" % (specificity))
  print("F1 Score: %.9f" % (score_f1))

##### Fungsi Bantuan untuk _Load Dataset_

In [6]:
def load_dataset():
	(trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()
	trainY = tf.keras.utils.to_categorical(trainY) ## Di ubah ke matrix persegi representasi binari
	testY = tf.keras.utils.to_categorical(testY) ## Di ubah ke matrix persegi representasi binari
	return trainX, trainY, testX, testY

#### Load Dataset

In [7]:
x_train, y_train, x_test, y_test = load_dataset()

#### Membangun Model _Fully Connected Neural Network_ dengan spesifikasi minimum memenuhi deskripsi Lab

In [8]:
## Melakukan inisialisasi node per layer
nodes_per_layer = [1000, 1000, 10]
activation_function_per_layer = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers = []
layers.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer)):
  node = nodes_per_layer[i]
  activation = activation_function_per_layer[i]
  layers.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model = Sequential(layers)

## Membangun model
model.build(input_shape=x_train.shape)
model.output_shape

(50000, 10)

#### Melakukan kompilasi ke dalam model

In [9]:
model.compile(
    optimizer=tf.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

#### Melakukan Fitting

In [10]:
beginning_hist = model.fit(x_train, y_train, epochs=10, batch_size=5000, validation_data=(x_test,y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 16s 2s/step - loss: 1628.5453 - accuracy: 0.1072 - recall: 0.1072 - specificity: 0.9008 - f1_score: 0.1072 - val_loss: 900.7686 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 2/10
10/10 [==============================] - 16s 2s/step - loss: 271.6966 - accuracy: 0.1099 - recall: 0.1099 - specificity: 0.9011 - f1_score: 0.1099 - val_loss: 65.4536 - val_accuracy: 0.0983 - val_recall: 0.0979 - val_specificity: 0.8999 - val_f1_score: 0.0980
Epoch 3/10
10/10 [==============================] - 15s 2s/step - loss: 42.3713 - accuracy: 0.1222 - recall: 0.1215 - specificity: 0.9033 - f1_score: 0.1220 - val_loss: 22.3440 - val_accuracy: 0.1498 - val_recall: 0.1483 - val_specificity: 0.9068 - val_f1_score: 0.1492
Epoch 4/10
10/10 [==============================] - 15s 2s/step - loss: 12.5823 - accuracy: 0.1404 - recall: 0.1329 - specificity: 0.9107 - f1_score: 0.1371 - val_loss: 4.5477 - val_accu

#### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [26]:
predict_and_print_metrics(model, x_test, y_test)

Accuracy: 0.101099998
Sensitivity: 0.000000000
Specificity: 0.999944508
F1 Score: 0.000000000


2. Anda dapat memperbaiki hasil yang diperoleh dengan melakukan  (tiga) skenario eksperimen, yaitu: _feature engineering_, menambahkan _layer_, dan _hyperparameter tuning_ (pada classifier).

#### _Feature Engineering_

- Untuk _feature engineering_ pada sesi lab kali ini, akan dilakukan normalisasi input agar berada pada rentang $[0...1]$.

In [38]:
def normalize(x):
  return x / 255.0

normalize_x_train = normalize(x_train)
normalize_x_test = normalize(x_test)

##### Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen _Feature Engineering_

In [39]:
## Melakukan inisialisasi node per layer
nodes_per_layer_for_feature_engineering = [1000, 1000, 10]
activation_function_per_layer_for_feature_engineering = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers_for_feature_engineering = []
layers_for_feature_engineering.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer_for_feature_engineering)):
  node = nodes_per_layer_for_feature_engineering[i]
  activation = activation_function_per_layer_for_feature_engineering[i]
  layers_for_feature_engineering.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model_for_feature_engineering = Sequential(layers_for_feature_engineering)

## Membangun model
model_for_feature_engineering.build(input_shape=x_train.shape)
model_for_feature_engineering.output_shape

(50000, 10)

##### Melakukan kompilasi ke dalam model

In [40]:
model_for_feature_engineering.compile(
    optimizer=tf.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

##### Melakukan _fitting_

In [41]:
feature_engineering_hist = model_for_feature_engineering.fit(normalize_x_train, y_train, epochs=10, batch_size=5000, validation_data=(normalize_x_test, y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 15s 2s/step - loss: 3.6769 - accuracy: 0.1210 - recall: 0.0182 - specificity: 0.9867 - f1_score: 0.0250 - val_loss: 2.3240 - val_accuracy: 0.1288 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 15s 1s/step - loss: 2.2371 - accuracy: 0.1627 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.1277 - val_accuracy: 0.2018 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 15s 1s/step - loss: 2.0700 - accuracy: 0.2424 - recall: 9.8000e-04 - specificity: 0.9999 - f1_score: 0.0019 - val_loss: 2.0168 - val_accuracy: 0.2758 - val_recall: 0.0080 - val_specificity: 0.9994 - val_f1_score: 0.0158
Epoch 4/10
10/10 [==============================] - 15s 1s/step - loss: 1.9709 - accuracy: 0.2896 - recall: 0.0117 - specificity: 0.9990 - f1_score: 0.0228 - val_loss: 

#### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [42]:
predict_and_print_metrics(model_for_feature_engineering, normalize_x_test, y_test)

Accuracy: 0.405299991
Sensitivity: 0.129992008
Specificity: 0.992755175
F1 Score: 0.213728368


#### Menambahkan _Layer_

- Untuk eksperimen menambahkan _layer_ pada sesi laboratorium kali ini, saya akan menambahkan layer dua buah _hidden layer_ lagi dengan jumlah _node_ sama dengan _hidden layer_ yang lain.

##### Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen Menambahkan _Layer_

In [11]:
## Melakukan inisialisasi node per layer
nodes_per_new_layer = [1000, 1000, 1000, 1000, 10]
activation_function_per_new_layer = [tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
new_layers = []
new_layers.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_new_layer)):
  node = nodes_per_new_layer[i]
  activation = activation_function_per_new_layer[i]
  new_layers.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
new_model = Sequential(new_layers)

## Membangun model
new_model.build(input_shape=x_train.shape)
new_model.output_shape

(50000, 10)

##### Melakukan kompilasi ke dalam model

In [14]:
new_model.compile(
    optimizer=tf.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

##### Melakukan _fitting_

In [15]:
new_model_hist = new_model.fit(x_train, y_train, epochs=10, batch_size=5000, validation_data=(x_test, y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 25s 3s/step - loss: 518.4105 - accuracy: 0.0992 - recall: 0.0991 - specificity: 0.9000 - f1_score: 0.0992 - val_loss: 117.4641 - val_accuracy: 0.1032 - val_recall: 0.1032 - val_specificity: 0.9004 - val_f1_score: 0.1032
Epoch 2/10
10/10 [==============================] - 25s 2s/step - loss: 99.3295 - accuracy: 0.1021 - recall: 0.1019 - specificity: 0.9005 - f1_score: 0.1020 - val_loss: 46.6571 - val_accuracy: 0.1027 - val_recall: 0.1027 - val_specificity: 0.9003 - val_f1_score: 0.1027
Epoch 3/10
10/10 [==============================] - 25s 3s/step - loss: 30.8576 - accuracy: 0.1068 - recall: 0.1062 - specificity: 0.9014 - f1_score: 0.1065 - val_loss: 7.8620 - val_accuracy: 0.1195 - val_recall: 0.1049 - val_specificity: 0.9178 - val_f1_score: 0.1138
Epoch 4/10
10/10 [==============================] - 25s 2s/step - loss: 7.9246 - accuracy: 0.1112 - recall: 0.1008 - specificity: 0.9127 - f1_score: 0.1066 - val_loss: 4.6546 - val_accuracy

##### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [30]:
predict_and_print_metrics(new_model, x_test, y_test)

Accuracy: 0.338299990
Sensitivity: 0.091553517
Specificity: 0.993520021
F1 Score: 0.155747622


#### _Hyperparameter Tuning_

In [45]:
## Melakukan inisialisasi node per layer
nodes_per_layer_for_hyperparameter = [1000, 1000, 10]
activation_function_per_layer_for_hyperparameter = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers_for_hyperparameter = []
layers_for_hyperparameter.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer_for_hyperparameter)):
  node = nodes_per_layer_for_hyperparameter[i]
  activation = activation_function_per_layer_for_hyperparameter[i]
  layers_for_hyperparameter.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model_for_hyperparameter = Sequential(layers_for_hyperparameter)

## Membangun model
model_for_hyperparameter.build(input_shape=x_train.shape)
model_for_hyperparameter.output_shape

(50000, 10)

In [49]:
model_for_hyperparameter.compile(
    optimizer=tf.optimizers.Adam(0.005),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

In [50]:
model_for_hyperparameter_hist = model_for_hyperparameter.fit(x_train, y_train, epochs=20, batch_size=10000, validation_data=(x_test, y_test), use_multiprocessing=True)

Epoch 1/20
5/5 [==============================] - 14s 3s/step - loss: 16854.2305 - accuracy: 0.0992 - recall: 0.0992 - specificity: 0.8999 - f1_score: 0.0992 - val_loss: 13292.7100 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 2/20
5/5 [==============================] - 14s 3s/step - loss: 4728.6680 - accuracy: 0.1001 - recall: 0.1001 - specificity: 0.9000 - f1_score: 0.1001 - val_loss: 4.6146 - val_accuracy: 0.1000 - val_recall: 0.0225 - val_specificity: 0.9787 - val_f1_score: 0.0370
Epoch 3/20
5/5 [==============================] - 14s 3s/step - loss: 2.7818 - accuracy: 0.1000 - recall: 0.0046 - specificity: 0.9955 - f1_score: 0.0076 - val_loss: 2.3038 - val_accuracy: 0.1001 - val_recall: 1.0000e-04 - val_specificity: 1.0000 - val_f1_score: 1.9994e-04
Epoch 4/20
5/5 [==============================] - 14s 3s/step - loss: 2.3049 - accuracy: 0.1000 - recall: 2.0000e-05 - specificity: 1.0000 - f1_score: 3.9996e-05 - val_loss: 2.3027 - 